In [44]:
from libraries import *

In [45]:
df = pd.read_csv("1-15Data-cleaned.csv")
df

,Section,Url,Content,Metadata
0,10. -Front Matter,https://uscode.house.gov/view.xhtml?req=granul...,18 USC Ch. 10: Front Matter Result ...,"Amendments2004-Pub. L. 108Ã¢ÂÂ458, title VI,..."
1,175. Prohibitions with respect to biological w...,https://uscode.house.gov/view.xhtml?req=granul...,"(a) In General.-Whoever knowingly develops, pr...",Amendments2002-Subsec. (c). Pub. L. 107Ã¢ÂÂ1...
2,175. a. Requests for military assistance to en...,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175a: Requests for military assistance ...,"References in TextSection 382 of title 10, ref..."
3,175. b. Possession by restricted persons,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175b: Possession by restricted persons ...,References in TextSection 351A of the Public H...
4,175. c. Variola virus,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175c: Variola virus Result 1...,NaN
...,...,...,...,...
141,289. False claims for pensions,https://uscode.house.gov/view.xhtml?req=granul...,"Whoever knowingly and willfully makes, or pres...",Amendments1994-Pub. L. 103Ã¢ÂÂ322 substitute...
142,290. Discharge papers withheld by claim agent,https://uscode.house.gov/view.xhtml?req=granul...,"Whoever, being a claim agent, attorney, or oth...",Amendments1994-Pub. L. 103Ã¢ÂÂ322 substitute...
143,291. Purchase of claims for fees by court offi...,https://uscode.house.gov/view.xhtml?req=granul...,"Whoever, being a judge, clerk, or deputy clerk...",Amendments1994-Pub. L. 103Ã¢ÂÂ322 substitute...
144,292. Solicitation of employment and receipt of...,https://uscode.house.gov/view.xhtml?req=granul...,Whoever solicits employment for himself or ano...,Amendments1994-Pub. L. 103Ã¢ÂÂ322 substitute...


In [3]:
df['Content'] = df['Content'].fillna("").astype(str)

In [81]:
for i in range(len(df['Content'])):
    if type(df['Content'][i]) == float:
        print(i)

47


In [82]:
max = 0
for i in range(len(df['Content'])):
    try:
        if len(df['Content'][i]) > max:
            max = len(df['Content'][i])
    except:
        print(i)

        
max

47


80533

In [46]:
print(len(df)) 

146


In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased") # nlpaueb/legal-bert-base-uncased   |  
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 3: Function to get embeddings
def get_embedding(text, tokenizer, model):
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    
    # Compute embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the mean of the token embeddings (for sentence embedding)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# # Step 4: Encode the entire 'Content' column
# embeddings = np.array([get_embedding(text, tokenizer, model).cpu().numpy() for text in df['Content']]).astype('float32')

# # Step 5: Initialize a FAISS index with L2 (Euclidean) distance
# dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings
# index = faiss.IndexFlatL2(dimension)  # IndexFlatL2 is a flat index (no compression)

# # Step 6: Add the vectors to the index
# index.add(embeddings)

# # Optional: Verify the number of vectors in the index
# print(f"Number of vectors in the index: {index.ntotal}")

# # Step 7: Prepare a test query

In [11]:
#np.savetxt("embeddings-legal-bert-base-uncased.csv", embeddings, delimiter=',')

In [47]:
embeddings = np.loadtxt("embeddings-legal-bert-base-uncased.csv", delimiter=',')

In [48]:
embeddings
embeddings.shape

(146, 768)

In [64]:
# if client.get_collection(collection_name):
#     client.delete_collection(collection_name)  # This will delete the collection if it exists
#     print(f"Collection '{collection_name}' deleted.")

Collection 'LawEmbedding' deleted.


In [12]:
# test = ''' 
#  for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''

# # Step 8: Encode the test query
# test_embedding = get_embedding(test, tokenizer, model).reshape(1, -1).cpu().numpy().astype('float32')

# # Step 9: Example: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 10: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])

: 

Cosine Similarity for FIASS

In [ ]:
# import faiss
# import numpy as np

# # Assuming 'embeddings' contains the embeddings loaded from your CSV file
# # Step 1: Normalize the embeddings for cosine similarity
# def normalize(embeddings):
#     norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
#     return embeddings / norms

# # Step 2: Normalize the existing embeddings
# normalized_embeddings = normalize(embeddings)

# # Step 3: Initialize a FAISS index with Inner Product (for cosine similarity)
# dimension = normalized_embeddings.shape[1]
# index = faiss.IndexFlatIP(dimension)  # IndexFlatIP is for inner product (dot product)

# # Step 4: Add the normalized vectors to the index
# index.add(normalized_embeddings)

# # Optional: Verify the number of vectors in the index
# print(f"Number of vectors in the index: {index.ntotal}")

# # Step 5: Normalize the test query embedding
# test = ''' 
# for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''
# # Get the test embedding using the existing model and tokenizer
# test_embedding = get_embedding(test, tokenizer, model).cpu().numpy().astype('float32')
# test_embedding = normalize(test_embedding.reshape(1, -1))

# # Step 6: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 7: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])


In [67]:
client = QdrantClient(
    url="https://cecd63ca-9699-4bc0-a60f-3c7e8c768fd3.europe-west3-0.gcp.cloud.qdrant.io:6333",  # Replace with your cloud cluster URL
    api_key="jwv0ETSb8751Q1yrQ3RVSsChFudsv1M63mEvxXXRnLf_ROoeAfe_Wg"  # Replace with your API key
)

collection_name = "LawEmbedding"

In [68]:
dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dimension, distance=Distance.COSINE)  # Use COSINE distance for similarity
)

# Step 5: Insert embeddings row by row into Qdrant
points = [
    {
        "id": idx,  # The row index will be used as the unique ID
        "vector": embeddings.tolist(),  # Convert numpy array to list for Qdrant
        "payload": {
            "section": df['Section'].iloc[idx],
            "url": df['Url'].iloc[idx],
        }
    }
    for idx, embeddings in enumerate(embeddings)
]

client.upsert(
    collection_name=collection_name,
    points=points
)

# Optional: Verify the number of vectors in the collection
info = client.get_collection(collection_name)
print(f"Number of vectors in the collection: {info.points_count}")

C:\Users\npatel237\AppData\Local\Temp\ipykernel_22308\2309866623.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Number of vectors in the collection: 146


In [71]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

test = ''' 
Mandatory Restitution.-Upon a conviction under this section, the court shall order restitution under
section 3663A in an amount equal to the total unpaid support obligation as it exists at the time of sentencing.
'''

test_embedding = get_embedding(test, tokenizer, model).cpu().numpy().astype('float32')
test_embedding_flat = test_embedding.flatten().tolist()  # Flatten to 1D list

search_result = client.search(
    collection_name=collection_name,
    query_vector=test_embedding_flat,  # Use 'query_vector' parameter directly
    limit=2  # Number of nearest neighbors you want to retrieve
)

# Step 5: Print the distances and indices of the nearest neighbors
for result in search_result:
    print(f"Distance: {result.score}")  # Score represents similarity (lower is better)
    print(f"Matched Title: {result.id}")
    print(f"Matched Url: {result.payload['url']}")
    print(f"Matched Section: {result.payload['section']}")

Distance: 0.90716904
Matched Title: 9
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section228&num=0&edition=prelim
Matched Section: 228. Failure to pay legal child support obligations
Distance: 0.89961714
Matched Title: 12
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section229A&num=0&edition=prelim
Matched Section: 229A. Penalties


In [63]:
print(df['Content'][12])
print(df['Content'][9])


(a) Criminal Penalties.-(1) In general.-Any person who violates section 229 of this title shall be fined under this title, or imprisoned for any term of years, or both.(2) Death penalty.-Any person who violates section 229 of this title and by whose action the death of another person is the result shall be punished by death or imprisoned for life.(b) Civil Penalties.-(1) In general.-The Attorney General may bring a civil action in the appropriate United States district court against any person who violates section 229 of this title and, upon proof of such violation by a preponderance of the evidence, such person shall be subject to pay a civil penalty in an amount not to exceed $100,000 for each such violation.(2) Relation to other proceedings.-The imposition of a civil penalty under this subsection does not preclude any other criminal or civil statutory, common law, or administrative remedy, which is available by law to the United States or any other person.(c) Reimbursement of Costs.